## Presentation Type Classifier and Networks


### What can You do with this Notebook

- Find and classify imitative points (fugas, imitative duos, and periodic entries)
- Build a network of patterns found in several pieces, and reveal 'communities' of compositions


#### Network Visualization with Louvain Communities

Read more about **Louvain Community Detection Algorithms**:  https://towardsdatascience.com/community-detection-algorithms-9bd8951e7dae

**Updated Summer 2022**
**Key Features**:

- Uses `getDistance` to identify `close matches` with side-by-side comparison of soggetti.  
- For example: With a distance of "1", the soggetti `4, 1, 2, 3`, and `5, 1, 2, 3` will count as the same. These are reported as "flexed entries" in a separate column.
- Finds time intervals between entries (expressed as offsets, like `8.0, 4.0, 8.0`)
- Finds melodic intervals between first note of successive entries in each pattern (like `P-5, P-8`)
- Labels Fuga, PEn, and ID according to time intervals between entries. NIm not yet supported!
- If two entries are separated by more than 10 bars (80 offsets), the tool resets to a new pattern
- Counts number of entries
- Provides offset and measure/beat locations
- Sorts all presentation types by the order in which they appear in the piece
- Reports voice names of the entries, in order of their appearance
- Omits singleton soggetti (just one entry of a given motive in isolation)
    
- Read the documentation:  `print(piece.presentationTypes.__doc__)`

In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy as np
import itertools
from itertools import combinations
import networkx as nx
from community import community_louvain
from copy import deepcopy
import glob
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


## A.  Start by Finding Presentation Types in a Corpus of Pieces 

* The **CorpusBase** class is a convenient way to find patterns in any given list of pieces.
* The pieces are provided as a **list**, within square brackets and separated by commas.  
* The bracketed list is then contained within the parentheses of `CorpusBase()`
* For example: 


```prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'
piece_list = ['CRIM_Model_0019',
             'CRIM_Mass_0019_1',
             'CRIM_Mass_0019_2',
             'CRIM_Mass_0019_3',
             'CRIM_Mass_0019_4',
             'CRIM_Mass_0019_5']
suffix = '.mei'
corpus_list  = []
for piece in piece_list:
url = prefix + piece + suffix
    corpus_list.append(url)

```
Or for local files:

```

corpus_list = []
for name in glob.glob('Music_Files/*'):
    corpus_list.append(name)

```




* Read the documentation:  `print(CorpusBase.batch.__doc__)`



In [10]:
# remote corpus

corpus_list = []

prefix = 'https://crimproject.org/mei/'

piece_list = ['CRIM_Model_0019', 'CRIM_Mass_0019_1']
suffix = '.mei'

for piece in piece_list:
    url = prefix + piece + suffix
    corpus_list.append(url)


corpus_list

['https://crimproject.org/mei/CRIM_Model_0019.mei',
 'https://crimproject.org/mei/CRIM_Mass_0019_1.mei']

In [3]:
# only if loading local files
corpus_list = []
for name in glob.glob('Music_Files/*'):
    corpus_list.append(name)


### Now Run the PresentationType method on the Corpus

In [16]:
# This will need to be modified if using a corpus method different from the CRIM Project above.
final = pd.DataFrame()

for work in corpus_list:

    piece = importScore(work)   
    output = piece.presentationTypes(limit_to_entries = True,
                                     head_flex = 1,
                                     body_flex = 0,
                                     include_hidden_types = False,
                                     combine_unisons = True,
                                     melodic_ngram_length = 4)

    final['Validation'] = ""
    final['Comments'] = ""
    final = final.append(output, ignore_index=True)

# final.to_csv("corpus_classified.csv")
# final.to_csv("saved_csv/CRIM_Mass_0019_P_Types_hidden.csv")

In [17]:
final

,Validation,Comments,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries,Parallel_Entries,Parallel_Voice,Count_Non_Overlaps
0,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0, 11/3.0, 14/1.0, 1...","[P-5, P-4, P-5, P8, P5, P-8]","[0.0, 8.0, 40.0, 48.0, 84.0, 104.0, 128.0]","[(-3, 3, 2, -2)]","[8.0, 32.0, 8.0, 36.0, 20.0, 24.0]","[Cantus, Altus, Tenor, Bassus, Altus, Cantus, ...",FUGA,7.0,False,0.0,None,0.0
1,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,150.0,"[19/4.0, 21/4.0, 22/4.0, 25/4.0, 27/4.0, 29/4....","[P8, P-5, P-4, P-5, P8, P8, P-11]","[150.0, 166.0, 174.0, 198.0, 214.0, 230.0, 250...","[(-3, 2, 2, -2), (-2, 2, 2, -2)]","[16.0, 8.0, 24.0, 16.0, 16.0, 20.0, 24.0]","[Tenor, Cantus, Altus, Tenor, Bassus, Altus, C...",FUGA,8.0,True,0.0,None,0.0
2,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,298.0,"[38/2.0, 38/4.0, 42/2.0, 44/1.0, 46/2.0, 48/4.0]","[P-5, P-5, P8, P8, P-11]","[298.0, 302.0, 330.0, 344.0, 362.0, 382.0]","[(-3, 3, -2, -2)]","[4.0, 28.0, 14.0, 18.0, 20.0]","[Altus, Tenor, Bassus, Altus, Cantus, Bassus]",FUGA,6.0,False,0.0,None,0.0
3,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,394.0,"[50/2.0, 51/4.0, 53/2.0, 55/2.0, 56/4.0, 59/1....","[P5, P-8, P4, P5, P-8, P8, P-4, P-5]","[394.0, 406.0, 418.0, 434.0, 446.0, 464.0, 486...","[(-2, 2, 3, -2)]","[12.0, 12.0, 16.0, 12.0, 18.0, 22.0, 4.0, 16.0]","[Altus, Cantus, Bassus, Tenor, Altus, Bassus, ...",FUGA,9.0,False,0.0,None,0.0
4,NaN,NaN,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0]","[P-5, P-4, P-5]","[0.0, 8.0, 40.0, 48.0]","[(-3, 3, 2, -2)]","[8.0, 32.0, 8.0]","[Cantus, Altus, Tenor, Bassus]",ID,4.0,False,0.0,None,0.0
5,NaN,NaN,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,78.0,"[10/4.0, 12/2.0, 14/2.0, 15/3.0]","[P-4, P-5, P-4]","[78.0, 90.0, 106.0, 116.0]","[(2, 2, 2, 2)]","[12.0, 16.0, 10.0]","[Cantus, Altus, Tenor, Bassus]",FUGA,4.0,False,0.0,None,0.0
6,NaN,NaN,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,160.0,"[20/1.0, 24/1.0, 30/2.0, 33/1.0, 35/3.0, 39/3.0]","[P-8, P1, P4, P-8, P5]","[160.0, 192.0, 242.0, 264.0, 284.0, 316.0]","[(-3, 2, 2, -2)]","[32.0, 50.0, 22.0, 20.0, 32.0]","[Cantus, Tenor, Bassus, Altus, Bassus, Bassus]",FUGA,6.0,False,0.0,None,2.0
7,NaN,NaN,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,164.0,"[20/3.0, 24/3.0, 27/2.0, 27/4.0, 30/4.0, 36/1....","[P-8, P8, P-8, P5, P4, P5, M-6]","[164.0, 196.0, 218.0, 222.0, 246.0, 288.0, 322...","[(2, 2, -2, -3)]","[32.0, 22.0, 4.0, 24.0, 42.0, 34.0, 4.0]","[Altus, Bassus, Cantus, Tenor, Tenor, Altus, C...",FUGA,8.0,False,0.0,None,1.0
8,NaN,NaN,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,372.0,"[45/3.0, 47/3.0, 49/3.0, 51/4.0, 53/3.0]","[P8, P-8, P8, P-12]","[372.0, 388.0, 404.0, 422.0, 436.0]","[(-3, 3, -2, -2)]","[16.0, 16.0, 18.0, 14.0]","[Tenor, Cantus, Bassus, Altus, Bassus]",FUGA,5.0,False,0.0,None,0.0
9,NaN,NaN,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,458.0,"[56/2.0, 58/4.0, 61/3.0, 63/2.0]","[P-8, P-5, P12]","[458.0, 478.0, 500.0, 514.0]","[(-2, 2, 3, -2)]","[20.0, 22.0, 14.0]","[Cantus, Tenor, Bassus, Cantus]",FUGA,4.0,False,0.0,None,0.0


## Various Options to Filter to Prepare to Network

See the Presentation Type Filter Notebook for more ideas!

But the results of this filtering are needed to make useful networks below!

* by type
* by piece
* according to length between entries

In [13]:
# filter for only ID or PEN
pen_id_filter = final['Presentation_Type'].isin(["PEN"])
pen_id_only = final[pen_id_filter]
final = pen_id_only.copy()
len(final)



0

## B. Network Visualization with Louvain Communities

* This will only make sense for a corpus!

## Add Combined Data for Melodic, Time and Soggetti as "Words" for Network Edges

- We combine the lists of time and melodic intervals of entries as strings (thus a kind of word that uniquely identifies that pattern)
- We do the same for soggetti, combining the melodic intervals themselves as a string
- These are in turn assembled in various combinations:  TINT, MINT, and SOG, also as strings

In [18]:
final["MINT"] = final["Melodic_Entry_Intervals"].apply(joiner)
final["TINT"] = final["Time_Entry_Intervals"].apply(joiner)

final['SOG'] = final['Soggetti'].apply(clean_melody_new)
final['ALL'] = final["MINT"] + '_' + final["TINT"] + '_' + final['SOG']

final["ALL_INT"] = final["MINT"] + '_' + final["TINT"]
final["MINT_SOG"] = final["MINT"] + '_' + final["SOG"]
final["TINT_SOG"] = final["TINT"] + '_' + final["SOG"]
final.head()

,Validation,Comments,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,...,Parallel_Entries,Parallel_Voice,Count_Non_Overlaps,MINT,TINT,SOG,ALL,ALL_INT,MINT_SOG,TINT_SOG
0,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0, 11/3.0, 14/1.0, 1...","[P-5, P-4, P-5, P8, P5, P-8]","[0.0, 8.0, 40.0, 48.0, 84.0, 104.0, 128.0]","[(-3, 3, 2, -2)]","[8.0, 32.0, 8.0, 36.0, 20.0, 24.0]",...,0.0,None,0.0,P-5_P-4_P-5_P8_P5_P-8,8.0_32.0_8.0_36.0_20.0_24.0,-3_3_2_-2,P-5_P-4_P-5_P8_P5_P-8_8.0_32.0_8.0_36.0_20.0_2...,P-5_P-4_P-5_P8_P5_P-8_8.0_32.0_8.0_36.0_20.0_24.0,P-5_P-4_P-5_P8_P5_P-8_-3_3_2_-2,8.0_32.0_8.0_36.0_20.0_24.0_-3_3_2_-2
1,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,150.0,"[19/4.0, 21/4.0, 22/4.0, 25/4.0, 27/4.0, 29/4....","[P8, P-5, P-4, P-5, P8, P8, P-11]","[150.0, 166.0, 174.0, 198.0, 214.0, 230.0, 250...","[(-3, 2, 2, -2), (-2, 2, 2, -2)]","[16.0, 8.0, 24.0, 16.0, 16.0, 20.0, 24.0]",...,0.0,None,0.0,P8_P-5_P-4_P-5_P8_P8_P-11,16.0_8.0_24.0_16.0_16.0_20.0_24.0,-3_2_2_-2,P8_P-5_P-4_P-5_P8_P8_P-11_16.0_8.0_24.0_16.0_1...,P8_P-5_P-4_P-5_P8_P8_P-11_16.0_8.0_24.0_16.0_1...,P8_P-5_P-4_P-5_P8_P8_P-11_-3_2_2_-2,16.0_8.0_24.0_16.0_16.0_20.0_24.0_-3_2_2_-2
2,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,298.0,"[38/2.0, 38/4.0, 42/2.0, 44/1.0, 46/2.0, 48/4.0]","[P-5, P-5, P8, P8, P-11]","[298.0, 302.0, 330.0, 344.0, 362.0, 382.0]","[(-3, 3, -2, -2)]","[4.0, 28.0, 14.0, 18.0, 20.0]",...,0.0,None,0.0,P-5_P-5_P8_P8_P-11,4.0_28.0_14.0_18.0_20.0,-3_3_-2_-2,P-5_P-5_P8_P8_P-11_4.0_28.0_14.0_18.0_20.0_-3_...,P-5_P-5_P8_P8_P-11_4.0_28.0_14.0_18.0_20.0,P-5_P-5_P8_P8_P-11_-3_3_-2_-2,4.0_28.0_14.0_18.0_20.0_-3_3_-2_-2
3,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,394.0,"[50/2.0, 51/4.0, 53/2.0, 55/2.0, 56/4.0, 59/1....","[P5, P-8, P4, P5, P-8, P8, P-4, P-5]","[394.0, 406.0, 418.0, 434.0, 446.0, 464.0, 486...","[(-2, 2, 3, -2)]","[12.0, 12.0, 16.0, 12.0, 18.0, 22.0, 4.0, 16.0]",...,0.0,None,0.0,P5_P-8_P4_P5_P-8_P8_P-4_P-5,12.0_12.0_16.0_12.0_18.0_22.0_4.0_16.0,-2_2_3_-2,P5_P-8_P4_P5_P-8_P8_P-4_P-5_12.0_12.0_16.0_12....,P5_P-8_P4_P5_P-8_P8_P-4_P-5_12.0_12.0_16.0_12....,P5_P-8_P4_P5_P-8_P8_P-4_P-5_-2_2_3_-2,12.0_12.0_16.0_12.0_18.0_22.0_4.0_16.0_-2_2_3_-2
4,NaN,NaN,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0]","[P-5, P-4, P-5]","[0.0, 8.0, 40.0, 48.0]","[(-3, 3, 2, -2)]","[8.0, 32.0, 8.0]",...,0.0,None,0.0,P-5_P-4_P-5,8.0_32.0_8.0,-3_3_2_-2,P-5_P-4_P-5_8.0_32.0_8.0_-3_3_2_-2,P-5_P-4_P-5_8.0_32.0_8.0,P-5_P-4_P-5_-3_3_2_-2,8.0_32.0_8.0_-3_3_2_-2


### Make Groups, Based on Selected Criteria:

* MINT (the melodic intervals between imitative entries)
* TINT (the time intervals between imitative entries)
* SOG (the soggetti)
* Or any of the combinations:
    * ALL (MINT + TINT + Soggetti)
    * ALL_INT (MINT + TINT)
    * MINT_SOG (MINT + Soggetti)
    * TINT_SOG (TINT + Soggetti)

In [19]:

output_grouped = final.groupby(['Title'])['MINT'].apply(list).reset_index()
output_grouped


,Title,MINT
0,Missa Veni sponsa Christi: Kyrie,"[P-5_P-4_P-5, P-4_P-5_P-4, P-8_P1_P4_P-8_P5, P..."
1,Veni sponsa Christi,"[P-5_P-4_P-5_P8_P5_P-8, P8_P-5_P-4_P-5_P8_P8_P..."


## Group and Find Unique Pairs

- the pairing must be applied to columns that have more than one value, like Title, since these form the edges

In [20]:
pairs = output_grouped.MINT.apply(lambda x: list(combinations(x, 2)))
pairs2 = pairs.explode().dropna()
unique_pairs = pairs.explode().dropna().unique()
len(unique_pairs)

27

In [21]:
# define network functions.  Do not edit!

def add_communities(G):
    G = deepcopy(G)
    partition = community_louvain.best_partition(G)
    nx.set_node_attributes(G, partition, "group")
    return G

def create_node_html(node: str, source_df: pd.DataFrame, node_col: str):
    rows = source_df.loc[source_df[node_col] == node].itertuples()
    
    html_lis = []
    
    for r in rows:
        html_lis.append(f"""<li>Composer: {r.Composer}<br>
                                Title: {r.Title}<br>
                                Offset: {r.First_Offset}</li>"""
                       )
        
    html_ul = f"""<ul>{''.join(html_lis)}</ul>"""
        
    return html_ul


def add_nodes_from_edgelist(edge_list: list, 
                               source_df: pd.DataFrame, 
                               graph: nx.Graph,
                               node_col: str):
    
    graph = deepcopy(graph)
    
    node_list = pd.Series(edge_list).apply(pd.Series).stack().unique()
    
    for n in node_list:
        graph.add_node(n, title=create_node_html(n, source_df, node_col))
        
    return graph

def choose_network(df, chosen_word, file_name):
    
    if len(df) < 1:
        print("Sorry, No P Types to display")
    else:
        output_grouped = df.groupby(['Title'])[chosen_word].apply(list).reset_index()
        pairs = output_grouped[chosen_word].apply(lambda x: list(combinations(x, 2)))
        pairs2 = pairs.explode().dropna()
        unique_pairs = pairs.explode().dropna().unique()


        pyvis_graph = Network(notebook=True, width="1800", height="1400", bgcolor="black", font_color="white")
        G = nx.Graph()

        try:
            G = add_nodes_from_edgelist(edge_list=unique_pairs, source_df=final, graph=G, node_col=chosen_word)
        except Exception as e:
            print(e)


        G.add_edges_from(unique_pairs)
        G = add_communities(G)
        pyvis_graph.from_nx(G)
        pyvis_graph.show(file_name)

In [22]:
# Find your rendered network in the Jupyter Hub file list!

choose_network(final, 'MINT', 'Mod_Mass_Full_Sog_19.html')